In [28]:
import os
import pandas as pd
import numpy as np
import re
import datetime
import pickle
import json
import time
from nltk.tokenize import sent_tokenize
import nltk
from collections import Counter

from nltk.stem import WordNetLemmatizer,PorterStemmer
  
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

os.chdir('..')
print(os.getcwd())
from utils import get_fulltext,get_fname,fulltext_exists
os.chdir('./data_processing/')

/Users/yiweiluo/scientific-debates


In [29]:
df = pd.read_pickle('../data_scraping/dedup_combined_df.pkl')
df.head(10)

,url,title,date,domain,stance,topic,is_AP,year,pretty_domain,month
0,www.buzzfeednews.com/article/tasneemnashrulla/...,"""eat the babies"" viral video at aoc town hall ...",2019-10-04 00:00:00,buzzfeed,pro,cc,None,2019.0,Buzzfeed,10.0
1,www.buzzfeednews.com/article/passantino/extrem...,"""extremely likely"" global warming is man-made,...",2013-09-27 00:00:00,buzzfeed,pro,cc,None,2013.0,Buzzfeed,9.0
2,shotofprevention.com/2010/11/03/history-makes-...,"""history"" makes headlines with launch of new w...",2020-03-13 14:32:02,https://shotofprevention/,pro,vax,False,2020.0,Shot of Prevention,3.0
3,www.buzzfeednews.com/article/andrewkaczynski/i...,"""it's global warming, stupid"" - buzzfeed news",2012-11-01 00:00:00,buzzfeed,pro,cc,None,2012.0,Buzzfeed,11.0
4,www.buzzfeednews.com/article/tasneemnashrulla/...,"""japan dropped an atomic bomb on america durin...",2014-02-24 00:00:00,buzzfeed,pro,cc,None,2014.0,Buzzfeed,2.0
5,www.buzzfeednews.com/article/llevin/opinion-im...,"""look at my record, child"": joe biden showed m...",2019-10-31 00:00:00,buzzfeed,pro,cc,None,2019.0,Buzzfeed,10.0
6,www.breitbart.com/politics/2019/06/14/orourke-...,"""president o'rourke will end oil and gas lease...",2019-06-14 00:00:00,breitbart,anti,cc,None,2019.0,Breitbart,6.0
7,www.buzzfeednews.com/article/andrewkaczynski/s...,smoking doesnt kill and other great old opeds ...,2015-03-31 00:00:00,buzzfeed,pro,cc,None,2015.0,Buzzfeed,3.0
8,www.foxnews.com/world/100-carbon-tax-by-2030-c...,"$100 carbon tax by 2030 could save climate, sa...",2017-05-29 00:00:00,fox,anti,cc,None,2017.0,Fox,5.0
9,childrenshealthdefense.org/news/4-billion-and-...,$4 billion and growing: u.s. payouts for vacc...,2018-11-19 00:00:00,chd,anti,vax,None,2018.0,Children's Health Defense,11.0


In [4]:
df.shape

(44582, 10)

# Contents:
* [Define functions](#Define-functions)
* [Extract clausal complements with SpaCy](#Extract-embedded-clauses-with-SpaCy)

# Define functions

We add a neural coref resolution step to the default SpaCy pipeline: https://github.com/huggingface/neuralcoref

In [30]:
import spacy
nlp = spacy.load('en_core_web_sm')


# First way we can control a parameter
#neuralcoref.add_to_pipe(nlp, greedyness=0.75)
import neuralcoref
neuralcoref.add_to_pipe(nlp)

Helper functions for the main ```get_quotes``` function:

In [31]:
def is_good_verb_dep(dep):
    return dep[:3] == 'aux' or dep[:3] == 'adv' or dep == 'det' or dep == 'rel' or dep == 'prep' or dep[-3:] == 'obj' or dep[-3:] == 'mod' or dep == 'prt' or (dep[-4:] == 'comp' and dep != 'ccomp')

def is_good_subj_dep(dep):
    return dep != 'ccomp'

def is_ROOT(tok):
    return tok.dep_ == 'ROOT' or tok.dep_[-2:] == 'cl' or tok.dep_ == 'ccomp' or \
            (tok.dep_ == 'conj' and tok.head.dep_ == 'ROOT')

REL_PRONOUNS = set(['who', 'whom', 'whose', 'which', 'that'])
def is_rel_pronoun(tok):
    tok = tok.lower().strip()
    return tok in REL_PRONOUNS

In [32]:
def get_quotes(text):
    # Step 0. Run pipeline.
    doc = nlp(text)

    # Step 1. Figure out which tokens to tag w/ coreferring token
    to_annotate = {}
    for clust in doc._.coref_clusters:
        non_main_mentions = [x for x in clust.mentions if x != clust.main]
        for mention in non_main_mentions:
            start = mention.start
            to_annotate[start] = True
            end = mention.end
            if start+1 != end:
                for ix in range(start+1,end):
                    to_annotate[ix] = False
    to_annotate.update({ix: False for ix in range(len(doc)) if ix not in to_annotate})

    annotated_tokens = {}
    for token in doc:
        #print(token.i,token.text,token.head,token.dep_)
        if to_annotate[token.i]:
            #print('Main span:',token._.coref_clusters[0].main)
            # print('Main span start token_ix, end token_ix:',token._.coref_clusters[0].main.start,
            #      token._.coref_clusters[0].main.end)
            # print('\n')
            annotated_tokens[token.i] = token._.coref_clusters[0].main
        else:
            annotated_tokens[token.i] = None

    # Step 3. Go through each sentence in the doc, extract up to 1 quote from each sentence
    quote_objs = []
    for sent in doc.sents:
        #print(sent)

        VERBS = list(np.unique([token.head for token in sent if token.dep_ == 'ccomp']))
        #print(VERBS)

        # Extract everything else for each VERB
        for VERB in VERBS:
            #print("\nCcomp dependency found! For quoting verb '{}'".format(VERB))
            # Extract the rest of the quoting verb
            verb_deps = [x for x in VERB.children if is_good_verb_dep(x.dep_)]
            #print("\tFound children:",verb_deps)
            for x in verb_deps:
                new_children = [c for c in x.children if is_good_verb_dep(c.dep_)]
                #print("\tAdding children of {}:".format(x.text),new_children)
                verb_deps.extend(new_children)
                #print("\tUpdated verb deps:",verb_deps)

            # If verb's head is not itself (i.e., verb is not the ROOT),
            # recursively trace back to ROOT, then add all children of ROOT
            ROOT = VERB
            if is_ROOT(ROOT):
                pass
                #print("\n\tVerb '{}' is the ROOT.".format(ROOT))
                #verb_deps.append(ROOT)
            else:
                #print('\n\tFinding ROOT...')
                while not is_ROOT(ROOT):
                    ROOT = ROOT.head
                    #print('\t\tCurrent root:',ROOT)
                assert is_ROOT(ROOT)
                verb_deps.append(ROOT)

                #print("\tAdding children of ROOT...")
                root_deps = [x for x in ROOT.children if is_good_verb_dep(x.dep_)]
                #print("\tFound ROOT deps:",root_deps)
                for x in root_deps:
                    new_children = [c for c in x.children if is_good_verb_dep(c.dep_)]
                    #print("\tAdding children of {}:".format(x.text),new_children)
                    root_deps.extend(new_children)
                    #print("\tUpdated ROOT deps:",root_deps)

                #print("\tAdding ROOT deps to verb deps...")
                verb_deps.extend([x for x in root_deps if x != VERB and x not in verb_deps])
                #print("\tUpdated verb deps:",verb_deps)

            NEG,IS_NEG,neg_children = None,None,None
            SUBJECT,subj_children = None,None

            #print("\nLooking for SUBJECT and NEGATION(s)...")
            for child in ROOT.children:
                if child.dep_[:5] == 'nsubj' or child.dep_ == 'expl':
                    SUBJECT = child
                    #print("\tFound SUBJECT:",SUBJECT)
                    if SUBJECT.head.dep_ == 'relcl' and is_rel_pronoun(SUBJECT.text): # we're dealing with the subject of a rel clause
                        #print("\tFound quote inside a relative clause. Finding antecedent subject...")
                        SUBJECT = SUBJECT.head.head
                        #print("\tTrue subject:",SUBJECT)
                    #print("Subject token '{}' is in a coref cluster:".format(SUBJECT),SUBJECT._.in_coref)

                if child.dep_[:3] == 'neg':
                    NEG = child
                    verb_deps.append(NEG)
                    neg_children = [c for c in NEG.children if c != VERB]
                    #print("n\tAdding new NEG children:",neg_children)
                    for x in neg_children:
                        new_children = [c for c in x.children]
                        #print("\tNew NEG grandchildren:",new_children)
                        neg_children.extend(new_children)
                        #print("\tUpdated neg_children:",neg_children)
                    verb_deps.extend(neg_children)
                    #print("\tUpdated verb_deps:",verb_deps)
                    IS_NEG = VERB in NEG.head.children or VERB == NEG.head

            if SUBJECT is None and (ROOT.dep_ == 'acl' or ROOT.dep_ == 'advcl'):
                main_verb = ROOT.head
                #print([(c.text,c.dep_) for c in main_verb.children])
                SUBJS = [c for c in main_verb.children if c.dep_[:5] == 'nsubj' or c.dep_ == 'expl']
                SUBJECT = SUBJS[0] if len(SUBJS) > 0 else None

            # Get rest of subject tokens
            if SUBJECT is not None:
                #print("\nFound SUBJECT:",SUBJECT)
                #print("\tAdding children of SUBJECT...")
                subj_children = [c for c in SUBJECT.children if is_good_subj_dep(c.dep_)]
                #print("\tFound children:",subj_children)
                for x in subj_children:
                    new_children = [c for c in x.children if is_good_subj_dep(c.dep_)]
                    #print("\tAdding children of child {}:".format(x.text),new_children)
                    subj_children.extend(new_children)
                    #print("\tUpdated subject children:",subj_children)

            #print("\nFinding quote introduced by '{}'...".format(VERB))
            emb_main_verbs = [c for c in VERB.children if c.dep_ == 'ccomp']
            #print("\tMain verbs of embedded clause:",emb_main_verbs)
            #assert len(emb_main_verbs) <= 2
            for emb_main_verb in emb_main_verbs:
                #print("\tMain *verb*:",emb_main_verb)

                # Recursively get all children of main verb of embedded clause
                #print("\nRecursively getting children of main verb of embedded clause...\n")
                #print("*"*50)
                children_queue = [x for x in emb_main_verb.children]
                #print("\tAdding children of main verb:",children_queue)
                for x in children_queue:
                    new_children = [c for c in x.children]
                    #print("\tAdding children of {}:".format(x.text),new_children)
                    children_queue.extend(new_children)
                    #print("\tNew children queue:",children_queue)

                # Sort children and matrix verb to be in correct order
                # Replace w/ coreferring text as necessary
                children_and_indices_and_coref = [(c,c.i,annotated_tokens[c.i]) for c in children_queue+[emb_main_verb]]
                sorted_ = sorted(children_and_indices_and_coref,key=lambda x:x[1])
                sorted_verb_tokens = sorted([(c,c.i,annotated_tokens[c.i]) for c in verb_deps+[VERB]],key=lambda x:x[1])
                #print("\n\tSorted verb tokens:",sorted_verb_tokens)
                if SUBJECT is not None:
                    sorted_subj_tokens = sorted([(c,c.i,annotated_tokens[c.i]) for c in subj_children+[SUBJECT]],key=lambda x:x[1])
                    #print("\tSorted subject tokens:",sorted_subj_tokens)
                else:
                    sorted_subj_tokens = None
                if NEG is not None:
                    sorted_neg_tokens = sorted([(c,c.i,annotated_tokens[c.i]) for c in neg_children+[NEG]],key=lambda x:x[1])
                else:
                    sorted_neg_tokens = None

                dict_ = {'quote tokens':[tup[0] for tup in sorted_],
                        'quote tokens coref':[tup[0] if tup[-1] is None else tup[-1] for tup in sorted_],
                        'verb tokens':[tup[0] for tup in sorted_verb_tokens],
                        'verb tokens coref':[tup[0] if tup[-1] is None else tup[-1] for tup in sorted_verb_tokens],
                        'main verb':VERB,
                        'main verb coref':annotated_tokens[VERB.i] if annotated_tokens[VERB.i] else VERB,
                       'subject tokens':[tup[0] for tup in sorted_subj_tokens] if sorted_subj_tokens is not None else None,
                        'subject tokens coref':([tup[0] if tup[-1] is None else tup[-1] for tup in sorted_subj_tokens])
                                   if sorted_subj_tokens is not None else None,
                       'main subject':SUBJECT if SUBJECT is not None else None,
                        'main subject coref':annotated_tokens[SUBJECT.i] if SUBJECT is not None and annotated_tokens[SUBJECT.i] else
                                   (SUBJECT if SUBJECT is not None else None),
                       'neg tokens':[tup[0] for tup in sorted_neg_tokens] if sorted_neg_tokens is not None else None,
                       'main neg':NEG if NEG is not None else None,
                       'is neg':IS_NEG}

                res_ = {}
                res_['quote lemmas'] = [x.lemma_ for x in dict_['quote tokens']]
                res_['quote lemmas coref'] = [x.lemma_ for x in dict_['quote tokens coref']]
                res_['verb lemmas'] = [x.lemma_ for x in dict_['verb tokens']]
                res_['verb lemmas coref'] = [x.lemma_ for x in dict_['verb tokens coref']]
                res_['main verb lemma'] = dict_['main verb'].lemma_
                res_['main verb lemma coref'] = dict_['main verb coref'].lemma_
                res_['subject lemmas'] = [x.lemma_ for x in dict_['subject tokens']] if dict_['subject tokens'] is not None else None
                res_['subject lemmas coref'] = [x.lemma_ for x in dict_['subject tokens coref']] if dict_['subject tokens coref'] is not None else None
                res_['main subject lemma'] = dict_['main subject'].lemma_ if dict_['main subject'] is not None else None
                res_['main subject lemma coref'] = dict_['main subject coref'].lemma_ if dict_['main subject coref'] is not None else None
                res_['neg lemmas'] = [x.lemma_ for x in dict_['neg tokens']] if dict_['neg tokens'] is not None else None
                res_['main neg lemma'] = dict_['main neg'].norm_ if dict_['main neg'] is not None else None
                res_['quote text'] = [x.text for x in dict_['quote tokens']]
                res_['verb text'] = [x.text for x in dict_['verb tokens']]
                res_['main verb text'] = dict_['main verb'].text
                res_['subject text'] = [x.text for x in dict_['subject tokens']] if dict_['subject tokens'] is not None else None
                res_['main subject text'] = dict_['main subject'].text if dict_['main subject'] is not None else None
                res_['is neg'] = dict_['is neg']

                quote_objs.append(res_)

    return quote_objs

In [33]:
doc = nlp("A peer-reviewed study has found that humans are causing climate change.")

In [34]:
for tok in doc:
    print(tok.text,tok.dep_,tok.head.text)

A det study
peer npadvmod reviewed
- punct reviewed
reviewed amod study
study nsubj found
has aux found
found ROOT found
that mark causing
humans nsubj causing
are aux causing
causing ccomp found
climate compound change
change dobj causing
. punct found


In [35]:
get_quotes("A peer-reviewed study has found that humans are causing climate change.")

[{'quote lemmas': ['that', 'human', 'be', 'cause', 'climate', 'change'],
  'quote lemmas coref': ['that', 'human', 'be', 'cause', 'climate', 'change'],
  'verb lemmas': ['have', 'find'],
  'verb lemmas coref': ['have', 'find'],
  'main verb lemma': 'find',
  'main verb lemma coref': 'find',
  'subject lemmas': ['a', 'peer', '-', 'review', 'study'],
  'subject lemmas coref': ['a', 'peer', '-', 'review', 'study'],
  'main subject lemma': 'study',
  'main subject lemma coref': 'study',
  'neg lemmas': None,
  'main neg lemma': None,
  'quote text': ['that', 'humans', 'are', 'causing', 'climate', 'change'],
  'verb text': ['has', 'found'],
  'main verb text': 'found',
  'subject text': ['A', 'peer', '-', 'reviewed', 'study'],
  'main subject text': 'study',
  'is neg': None}]

# Extract clausal complements with SpaCy

(Did this part on the cluster.)

In [209]:
s = time.time()

In [210]:
time.time()-s

0.5713517665863037

## Missing URLs that cluster didn't get

In [141]:
missing_covid_urls = pickle.load(open('missing_urls.pkl','rb'))
len(missing_covid_urls)

60

In [151]:
covid_df = df.loc[df.topic=='covid']

In [152]:
start_time = time.time()
for url_ix in missing_covid_urls:
    curr_url = covid_df.url.values[url_ix]
    print(curr_url)
    quotes = get_quotes(get_fulltext(curr_url)[0])
    fname = get_fname(curr_url)
    with open('./extracted_quotes/{}.jl'.format(fname),'w+') as f:
        for res in quotes:
            json.dump(res, f)
            f.write('\n')
    if url_ix % 100 == 0:
        print('{}\t{}\tElapsed time in seconds:'.format(url_ix,fname),(time.time()-start_time)/60.)

www.dailymail.co.uk/news/article-7945897/Asylum-seekers-Christmas-Island-kept-away-coronavirus-evacuees.html?ns_mchannel=rss&ns_campaign=1490&ito=1490
6400	www.dailymail.co.uk[SEP]news[SEP]article-7945897[SEP]Asylum-seekers-Christmas-Island-kept-	Elapsed time in seconds: 0.014775717258453369
bipartisanreport.com/2020/03/18/biden-publicly-humiliates-trump-with-coronavirus-failure-fact-check/
redstate.com/stu-in-sd/2020/03/11/cartoons-and-kids-–-that’s-where-the-social-engineers-start-their-brainwashing/
bipartisanreport.com/2020/03/26/china-issues-rebuke-of-pompeos-wuhan-virus-moniker/
www.nytimes.com/2020/02/14/pageoneplus/corrections-feb-15-2020.html
www.dailymail.co.uk/news/article-8199959/Face-masks-NOT-stop-healthy-people-catching-coronavirus-says.html?ns_mchannel=rss&ns_campaign=1490&ito=1490
bipartisanreport.com/2020/03/23/fauci-reveals-he-cant-physically-control-trump-lies/
www.dailymail.co.uk/news/article-8172321/FBI-warned-concerns-biosecurity-risk-Chinese-scientists-research-

# Retrieval of extracted complement clauses from filename

In [4]:
QUOTES_DIR = './extracted_quotes/'

In [5]:
extracted_set = set(os.listdir(QUOTES_DIR))
len(extracted_set)

44527

In [6]:
for ix in range(0,len(df)):
    url = df.url.values[ix]
    fname = url.replace('/','[SEP]')
    assert '{}.jl'.format(fname) in extracted_set or \
    '{}.jl'.format(fname[:90]) in extracted_set

In [7]:
def get_quotes_from_file(url,df,missing):
    fname = url.replace('/','[SEP]')
    if fname+'.jl' in extracted_set:
        fname = fname+'.jl'
    else:
        fname = fname[:90]+'.jl'
    
    try:
        with open(os.path.join(QUOTES_DIR,fname),'r') as f:
            data = f.readlines()
        quotes = []
        for x in data:
            res = json.loads(x.strip())
            res.update({'source':url})                
            quotes.append(res)
            
        return quotes
    except FileNotFoundError:
        print(fname)
        missing.append(fname)

In [316]:
#len(os.listdir(DIR_WITH_NAMES))+len(os.listdir(DIR_WITH_NOS)),len(df)

In [200]:
missing = []
# all_quotes = [get_quotes_from_file(url_ix,df,missing) for url_ix in range(0,len(df))]
# all_quotes = [item for sublist in all_quotes for item in sublist]
covid_quotes = [get_quotes_from_file(url,df,missing) for url in covid_df.url.values]
#covid_quotes = [item for sublist in covid_quotes for item in sublist]

In [203]:
covid_quotes = [item for sublist in covid_quotes for item in sublist]
len(covid_quotes)

293234

In [205]:
covid_quotes[10]

{'quote lemmas': ['-PRON-', 'other', 'than', '-PRON-'],
 'quote lemmas coref': ['-PRON-', 'other', 'than', '-PRON-'],
 'verb lemmas': ['can', 'see'],
 'verb lemmas coref': ['can', 'see'],
 'main verb lemma': 'see',
 'main verb lemma coref': 'see',
 'subject lemmas': ['no', 'one'],
 'subject lemmas coref': ['no', 'one'],
 'main subject lemma': 'one',
 'main subject lemma coref': 'one',
 'neg lemmas': None,
 'main neg lemma': None,
 'quote text': ['him', 'other', 'than', 'me'],
 'verb text': ['can', 'see'],
 'main verb text': 'see',
 'subject text': ['No', 'one'],
 'main subject text': 'one',
 'is neg': None,
 'source': 'www.buzzfeednews.com/article/scaachikoul/animal-crossing-quarantine-distraction'}

In [206]:
len(missing),len(covid_quotes)

(0, 293234)

In [207]:
pickle.dump(covid_quotes,open('all_covid_quotes.pkl','wb'))

## Filter to true indirect statements

In [5]:
all_quotes = pickle.load(open('all_covid_quotes.pkl','rb'))

In [6]:
all_quotes[0].keys()

dict_keys(['quote lemmas', 'quote lemmas coref', 'verb lemmas', 'verb lemmas coref', 'main verb lemma', 'main verb lemma coref', 'subject lemmas', 'subject lemmas coref', 'main subject lemma', 'main subject lemma coref', 'neg lemmas', 'main neg lemma', 'quote text', 'verb text', 'main verb text', 'subject text', 'main subject text', 'is neg', 'source'])

In [9]:
counted_main_verb_lemmas = sorted(Counter([q['main verb lemma'] for q in all_quotes]).items(),key=lambda x:x[1],
                                  reverse=True)
counted_main_verb_lemmas

[('say', 135968),
 ('be', 47374),
 ('make', 25420),
 ('think', 20461),
 ('tell', 16615),
 ('know', 13915),
 ('find', 12468),
 ('show', 9905),
 ('believe', 9858),
 ('see', 8952),
 ('mean', 8234),
 ('let', 8160),
 ('suggest', 8059),
 ('help', 7236),
 ('have', 7217),
 ('note', 7006),
 ('allow', 6776),
 ('’s', 6667),
 ('argue', 6643),
 ('write', 6425),
 ('add', 6283),
 ('report', 5382),
 ('get', 5196),
 ('claim', 4505),
 ('ask', 4355),
 ('explain', 4337),
 ('want', 4332),
 ('—', 3704),
 ('warn', 3294),
 ('sure', 3243),
 ('announce', 3178),
 ('estimate', 3090),
 ('cause', 3078),
 ('conclude', 2959),
 ('understand', 2834),
 ('do', 2660),
 ('ensure', 2604),
 ('hope', 2601),
 ('expect', 2592),
 ('agree', 2275),
 ('consider', 2241),
 ('point', 2224),
 ('predict', 2142),
 ('state', 2099),
 ('indicate', 2027),
 ('’', 2013),
 ('declare', 1930),
 ('feel', 1876),
 ('assume', 1867),
 ('acknowledge', 1861),
 ('realize', 1842),
 ('insist', 1759),
 ('determine', 1736),
 ('decide', 1566),
 ('seem', 1442)

In [7]:
BLACKLIST_MAIN_VERBS = set(['make','let','help','allow','get','ask','want','cause','ensure','require','go',
                           'take','hold','become','watch','use','keep','call','give','come','change',
                           'look','remain','now','leave','include','begin','put','start','work','right',
                           'create','provide','produce','study','offer','try','lead','grow','like','set',
                           'follow','address','sign','affect','increase','render','rise','fall','pay',
                           'reflect','lose','stop','run','release','spend','build','choose','end','measure',
                           'involve','reduce','examine','bring','investigate','request','talk','identify',
                           'reach','move','explore','emerge','depend','send','occur','raise','die',
                           'drive','represent','vote','face','much','plan','dictate','meet','live',
                           'receive','bad','fail','stand','win','achieve','burn','push','cut','warm',
                           'exist','test','kill','cost','pass','result','cover','force','control','develop',
                           'check','love','protect','prevent','buy','break','seek','tend','emit','lay',
                           'hit','track','play','enable','capture','catch',
                            'have','do','decide','direct','need','succeed','hee','carry','dump','mean','ensure'])

In [8]:
def get_quotes_with_lemma(lemma_set,element,quote_list):
    return [q for q in quote_list if q[element] is not None and q[element].lower() in lemma_set]

def get_quotes_without_lemma(lemma_set,element,quote_list):
    return [q for q in quote_list if q[element] is not None and q[element].lower() not in lemma_set]

def get_quotes_with_stem(stem_set,element,quote_list):
    return [q for q in quote_list if q[element] is not None and ps.stem(q[element].lower()) in stem_set]

def get_quotes_without_stem(stem_set,element,quote_list):
    return [q for q in quote_list if q[element] is not None and ps.stem(q[element].lower()) not in stem_set]

In [9]:
# Remove quotes with question verb

QUESTION_WORDS = set(['who','what','when','where','why','how','if','whose','whether','which','whom','whence'])
QUESTION_VERBS = set(['ask','wonder','figure','guess','inquire','interrogate','question'])

def has_indirect_question(quote_obj):
    quote = quote_obj['quote']
    first_word = quote[0].lower()
    return first_word in QUESTION_WORDS

def has_question_verb(quote_obj):
    verb = quote_obj['main verb lemma'].lower()
    return verb in QUESTION_VERBS

In [10]:
all_quotes = [q for q in all_quotes if not has_question_verb(q)]
len(all_quotes)

289481

In [11]:
# Set aside quotes with 'be' as main verb

BE_STEMS = set(['is','wa','’s','are',"'s",'be','’m','am','are','’re',"'m","'re","been",'were'])

quotes_without_be = get_quotes_without_lemma(BE_STEMS,'main verb lemma',all_quotes)
quotes_with_be = get_quotes_with_lemma(BE_STEMS,'main verb lemma',all_quotes)

len(quotes_without_be),len(quotes_with_be)

(268236, 21245)

In [12]:
# Remove quotes with 'point','turn' but not 'point_out','turn_out' as main verb

point_quotes = get_quotes_with_lemma(['point','turn'],'main verb lemma',quotes_without_be)
point_out_quotes = [x for x in point_quotes if 'out' in x['verb lemmas']]

true_point_out_quotes = []
false_point_out_quotes = []
for x in point_out_quotes:
    indices_out = [i for i,t in enumerate(x['verb lemmas']) if t.lower() == 'out']
    prev_toks = [ps.stem(x['verb lemmas'][i-1].lower()) for i in indices_out]
    if 'point' in prev_toks:
        x['main verb lemma'] = 'point_out'
        x['main verb lemma coref'] = 'point_out'
        true_point_out_quotes.append(x)
    elif 'turn' in prev_toks:
        x['main verb lemma'] = 'turn_out'
        x['main verb lemma coref'] = 'turn_out'
        true_point_out_quotes.append(x)
    else:
        false_point_out_quotes.append(x)
        
len(true_point_out_quotes),len(false_point_out_quotes)

(735, 0)

In [13]:
all_quotes = [q for q in quotes_without_be if q not in false_point_out_quotes]
len(all_quotes)

268236

In [14]:
# Remove quotes with a set of non-attributive verbs introducing comp clause
non_other_verb_quotes = get_quotes_without_stem(BLACKLIST_MAIN_VERBS,'main verb lemma',all_quotes)
len(non_other_verb_quotes)

226381

In [15]:
# Remove quotes with 'seem', 'appear' as main verb

non_seem_quotes = get_quotes_without_stem(['seem','appear'],'main verb lemma',non_other_verb_quotes)
len(non_seem_quotes)#+len(seem_quotes),len(all_quotes)

225383

In [16]:
# Remove verb with non-alpha main verb

non_punc_main_verb_quotes = [q for q in non_seem_quotes if q['main verb lemma'][0].lower().isalpha()]
len(non_punc_main_verb_quotes)

223227

In [17]:
non_punc_main_verb_quotes[32432]

{'quote lemmas': ['-PRON-', 'ignore', 'expert', 'advice', ' '],
 'quote lemmas coref': ['President Donald Trump',
  'ignore',
  'expert',
  'advice',
  ' '],
 'verb lemmas': ['admit'],
 'verb lemmas coref': ['admit'],
 'main verb lemma': 'admit',
 'main verb lemma coref': 'admit',
 'subject lemmas': ['trump'],
 'subject lemmas coref': ['President Donald Trump'],
 'main subject lemma': 'trump',
 'main subject lemma coref': 'President Donald Trump',
 'neg lemmas': None,
 'main neg lemma': None,
 'quote text': ['he', 'ignores', 'expert', 'advice', ' '],
 'verb text': ['admits'],
 'main verb text': 'admits',
 'subject text': ['Trump'],
 'main subject text': 'Trump',
 'is neg': None,
 'source': 'www.vox.com/2020/3/24/21191557/trump-coronavirus-economy-reopen-social-distance'}

In [18]:
# Collapse recognize/recognise, realize/realise

british_quotes = [q for q in non_punc_main_verb_quotes if q['main verb lemma'].lower() == 'recognise' 
                      or q['main verb lemma'].lower() == 'realise']
print(len(british_quotes))
americanized_quotes = []
for q in british_quotes:
    q['main verb lemma'] = q['main verb lemma'].replace('s','z')
    q['main verb lemma coref'] = q['main verb lemma coref'].replace('s','z')
    americanized_quotes.append(q)
print(len(americanized_quotes))

154
154


In [19]:
spell_standardized_quotes = [q for q in non_punc_main_verb_quotes if q not in british_quotes] + americanized_quotes
len(spell_standardized_quotes)

223227

In [20]:
import re
re.sub("\s\s+",'','"   e&e')

'"e&e'

In [21]:
whitespace_cleaned_quotes = []
for q in spell_standardized_quotes:
    for key in ['main verb lemma','main verb lemma coref','main subject lemma','main subject lemma coref']:
        if q[key] is not None:
            q[key] = re.sub("\s\s+",'',q[key])
    whitespace_cleaned_quotes.append(q)

In [22]:
len(whitespace_cleaned_quotes)

223227

In [23]:
def get_src_attr(url,attr):
    return df.loc[df.url==url][attr].values[0]

In [24]:
updated_quotes = []
for q in whitespace_cleaned_quotes:
    stance = get_src_attr(q['source'],'stance')
    topic = get_src_attr(q['source'],'topic')
    q.update({'stance':stance,'topic':topic})
    updated_quotes.append(q)

In [25]:
len(updated_quotes)

223227

In [26]:
hop_collapsed_quotes = []
for q in updated_quotes:
    if q['main verb lemma'] == 'hop':
        q['main verb lemma coref'] = 'hope'
        q['main verb lemma'] = 'hope'
        q['verb lemmas'] = [v if v != 'hop' else 'hope' for v in q['verb lemmas']]
        q['verb lemmas coref'] = [v if v != 'hop' else 'hope' for v in q['verb lemmas coref']]
    hop_collapsed_quotes.append(q)
len(updated_quotes),len(hop_collapsed_quotes)

(223227, 223227)

In [27]:
pickle.dump(hop_collapsed_quotes,open('filtered_covid_quotes.pkl','wb'))

## Examine distribution of indirect statements

In [ ]:
# most common main verbs, subjects, amount of negation, sources

In [88]:
quotes = pickle.load(open('filtered_quotes.pkl','rb'))
len(quotes)

446764

In [89]:
all_main_verb_lemmas = [x['main verb lemma'].lower() for x in quotes]
counted_main_verb_lemmas = Counter(all_main_verb_lemmas)
sorted(counted_main_verb_lemmas.items(),key=lambda x:x[1],reverse=True)

[('say', 135970),
 ('think', 20463),
 ('tell', 16615),
 ('know', 13916),
 ('find', 12468),
 ('show', 9906),
 ('believe', 9859),
 ('see', 8952),
 ('mean', 8234),
 ('suggest', 8059),
 ('have', 7231),
 ('note', 7012),
 ('argue', 6643),
 ('write', 6425),
 ('add', 6283),
 ('report', 5383),
 ('claim', 4505),
 ('explain', 4337),
 ('warn', 3294),
 ('sure', 3243),
 ('announce', 3178),
 ('estimate', 3090),
 ('cause', 3078),
 ('conclude', 2959),
 ('understand', 2837),
 ('do', 2670),
 ('ensure', 2604),
 ('hope', 2602),
 ('expect', 2593),
 ('agree', 2276),
 ('consider', 2241),
 ('predict', 2142),
 ('state', 2099),
 ('point_out', 2055),
 ('indicate', 2027),
 ('realize', 1982),
 ('declare', 1930),
 ('feel', 1876),
 ('assume', 1869),
 ('acknowledge', 1861),
 ('insist', 1759),
 ('determine', 1736),
 ('decide', 1566),
 ('admit', 1430),
 ('matter', 1414),
 ('need', 1386),
 ('reveal', 1355),
 ('hear', 1335),
 ('learn', 1191),
 ('prove', 1147),
 ('remember', 1128),
 ('confirm', 1058),
 ('recognize', 1035),

In [90]:
'wonder' in counted_main_verb_lemmas

False

In [92]:
other_quotes = get_quotes_with_stem({
    'put'
},'main verb lemma',quotes)

In [93]:
other_quotes[:10]

[]

A lot of quotes don't have a subject:

In [100]:
len([q for q in quotes if q['main subject lemma coref'] is None])

41849

The most common subject stems--it looks like coref didn't fully do its job.

In [96]:
quotes[0].keys()

dict_keys(['quote lemmas', 'quote lemmas coref', 'verb lemmas', 'verb lemmas coref', 'main verb lemma', 'main verb lemma coref', 'subject lemmas', 'subject lemmas coref', 'main subject lemma', 'main subject lemma coref', 'neg lemmas', 'main neg lemma', 'quote text', 'verb text', 'main verb text', 'subject text', 'main subject text', 'is neg', 'source'])

In [101]:
all_main_subjs = [x['main subject lemma coref'].lower() for x in quotes if x['main subject lemma coref'] is not None]
counted_main_subjs = Counter(all_main_subjs)
sorted(counted_main_subjs.items(),key=lambda x:x[1],reverse=True)

[('-pron-', 76401),
 ('scientist', 6268),
 ('study', 5692),
 ('report', 4723),
 ('people', 3659),
 ('researcher', 3602),
 ('that', 3372),
 ('official', 3312),
 ('expert', 2712),
 ('this', 2231),
 ('trump', 2229),
 ('one', 1988),
 ('company', 1727),
 ('group', 1647),
 ('percent', 1532),
 ('research', 1508),
 ('agency', 1394),
 ('’s', 1322),
 ('author', 1311),
 ('some', 1285),
 ('government', 1106),
 ('obama', 1103),
 ('critic', 934),
 ('president', 915),
 ('analysis', 913),
 ('administration', 896),
 ('many', 851),
 ('poll', 826),
 ('team', 805),
 ('datum', 776),
 ('which', 760),
 ('other', 737),
 ('those', 731),
 ('court', 729),
 ('department', 723),
 ('analyst', 709),
 ('there', 701),
 ('environmentalist', 690),
 ('paper', 685),
 ('leader', 680),
 ('democrats', 670),
 ('country', 645),
 ('panel', 642),
 ('activist', 641),
 ('—', 639),
 ('change', 628),
 ('sanders', 610),
 ('“', 593),
 ('who', 572),
 ('times', 570),
 ('model', 567),
 ('evidence', 559),
 ('survey', 554),
 ('pruitt', 550

In [102]:
coref_failed_quotes = [x for x in quotes if x['main subject lemma coref'] is not None and 
                       x['main subject lemma coref'] == '-PRON-']
len(coref_failed_quotes)

76401

In [108]:
true_coref_failed_quotes = [x for x in coref_failed_quotes if x['main subject text'].lower() == 'he' 
                           or x['main subject text'].lower() == 'she' or 
                           x['main subject text'].lower() == 'they' or 
                           x['main subject text'].lower() == 'them' or 
                           x['main subject text'].lower() == 'her' or 
                           x['main subject text'].lower() == 'him']
len(true_coref_failed_quotes)

12130

In [112]:
good_quotes = [q for q in quotes if q not in true_coref_failed_quotes]
len(quotes),len(good_quotes)

(446764, 434634)

In [113]:
pickle.dump(good_quotes,open('good_coref_quotes.pkl','wb'))

In [105]:
coref_failed_quote_subjs = [x['main subject text'].lower() for x in coref_failed_quotes]
counted_coref_failed_subjs = Counter(coref_failed_quote_subjs)
sorted(counted_coref_failed_subjs.items(),key=lambda x:x[1],reverse=True)

[('i', 27709),
 ('we', 15663),
 ('you', 10443),
 ('it', 9422),
 ('he', 6631),
 ('she', 3325),
 ('they', 2125),
 ('us', 302),
 ('me', 278),
 ('’s', 196),
 ("'s", 129),
 ('them', 32),
 ('him', 12),
 ('myself', 8),
 ('ourselves', 7),
 ('china', 7),
 ('pruitt', 7),
 ('her', 5),
 ('gore', 5),
 ('baker', 4),
 ('yang', 4),
 ('bennet', 4),
 ('johnson', 4),
 ('yourself', 4),
 ('fox', 3),
 ('christy', 3),
 ('himself', 3),
 ('jackson', 3),
 ('fisher', 3),
 ('nasa', 3),
 ('kim', 3),
 ('sanders', 3),
 ('williams', 3),
 ('nour', 2),
 ('themselves', 2),
 ('inslee', 2),
 ('ours', 2),
 ('kirschner', 2),
 ('carla', 1),
 ('aep', 1),
 ('your', 1),
 ('lujan', 1),
 ('emily', 1),
 ('li', 1),
 ('our', 1),
 ('cai', 1),
 ('lau', 1),
 ('fischer', 1),
 ('alvarado', 1),
 ('mueller', 1),
 ('hvidovre', 1),
 ('rossiter', 1),
 ('claire', 1),
 ('clemente', 1),
 ('greta', 1),
 ('romney', 1),
 ('meyer', 1),
 ('amador', 1),
 ('innocent', 1),
 ('kam', 1),
 ('jones', 1),
 ('francis', 1),
 ('cnn', 1),
 ('evans', 1),
 ('tuttl

In [106]:
he_quotes = [x for x in coref_failed_quotes if x['main subject text'].lower() == 'he']
len(he_quotes)

6631

In [107]:
he_quotes[0]

{'quote lemmas': ['People',
  'free',
  '-',
  'ride',
  'when',
  '-PRON-',
  'jump',
  'the',
  'turnstile',
  'on',
  'the',
  'subway'],
 'quote lemmas coref': ['People',
  'free',
  '-',
  'ride',
  'when',
  'People',
  'jump',
  'the',
  'turnstile',
  'on',
  'the',
  'subway'],
 'verb lemmas': ['say'],
 'verb lemmas coref': ['say'],
 'main verb lemma': 'say',
 'main verb lemma coref': 'say',
 'subject lemmas': ['-PRON-'],
 'subject lemmas coref': ['-PRON-'],
 'main subject lemma': '-PRON-',
 'main subject lemma coref': '-PRON-',
 'neg lemmas': None,
 'main neg lemma': None,
 'quote text': ['People',
  'free',
  '-',
  'ride',
  'when',
  'they',
  'jump',
  'the',
  'turnstile',
  'on',
  'the',
  'subway'],
 'verb text': ['said'],
 'main verb text': 'said',
 'subject text': ['he'],
 'main subject text': 'he',
 'is neg': None,
 'source': 'www.washingtonpost.com/national/health-science/a-kind-of-dark-realism-why-the-climate-change-problem-is-starting-to-look-too-big-to-solve/20

Somewhere between 9.1k to 9.6k quotes have a negated main verb.

In [114]:
len([q for q in good_quotes if q['is neg']])

9155

In [116]:
len([q for q in good_quotes if q['main neg lemma'] is not None])

9629

In [117]:
len(good_quotes)-9629

425005

In [118]:
all_quote_domains = [df.loc[df.url==q['source']]['pretty_domain'].values[0] for q in good_quotes]
# counted_quote_domains = Counter(all_quote_domains)
# sorted(counted_quote_domains.items(),key=lambda x:x[1],reverse=True)

In [119]:
counted_quote_domains = Counter(all_quote_domains)
sorted(counted_quote_domains.items(),key=lambda x:x[1],reverse=True)

[('New York Times', 111781),
 ('Mother Jones', 43489),
 ('Fox', 34778),
 ('Washington Post', 29123),
 ('Christian Science Monitor', 29030),
 ('Breitbart', 22694),
 ('Guardian (US)', 22631),
 ('The Nation', 22467),
 ('Vox', 18943),
 ('Buzzfeed', 9161),
 ('Daily Caller', 8802),
 ('Grist', 8364),
 ("Children's Health Defense", 7836),
 ('Shot of Prevention', 7310),
 ('Alternet', 5988),
 ('Blaze', 5907),
 ('National Review', 5089),
 ('PJ Media', 4953),
 ('Drudgereport', 3752),
 ('USA Today', 3243),
 ('Reason', 2417),
 ('Newsweek', 2278),
 ('Quartz', 2214),
 ('The Progressive', 1822),
 ('Voices for Vaccines', 1775),
 ('The Verge', 1746),
 ('American Thinker', 1442),
 ('Democracy Now', 1398),
 ('NBC', 1313),
 ('CNS', 1309),
 ('Redstate', 1102),
 ('Sgtreport', 1095),
 ('Activist Post', 1089),
 ('Vice', 1059),
 ('New York Magazine', 1041),
 ('Infowars', 781),
 ('In These Times', 752),
 ('News With Views', 607),
 ('Citizens', 564),
 ('Hot Air', 505),
 ('The Week', 472),
 ('The American Spectator

# Additional pre-processing

## Add NER tags?